<a href="https://colab.research.google.com/github/iceman67/-Python/blob/master/kepco_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

전력 사용 데이터셋 사용하기



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error # 정확도 평가
import lightgbm as lgb
import math

pd.set_option('display.max_columns', 15)

In [ ]:
def data_read(name): # 입력 데이터 읽기
    df = pd.read_csv(f'{name}.csv', encoding='cp949')
    df = df.set_index('date_time')
    df.columns = ['num', 'pcon', 'temp', 'wind', 'humidity', 'rain', 'sunshine', 'non-elec', 'pv']

    df['dt'] = df.index # index 복사
    df['dt'] = pd.to_datetime(df['dt'], format='%Y-%m-%d %H') # datetime 변환, 복사

    df['dayofweek'] = df['dt'].dt.dayofweek # 요일 추가
    df['month'] = df['dt'].dt.month # 월 추가
    df['hour'] = df['dt'].dt.hour # 시간

    df = df.drop(['dt'], axis=1)
    dfc = df.copy() # 메모리를 달리해서 dataframe 복사

    dfc['non-elec'] = dfc['non-elec'].fillna(0) # nan을 0으로 채움
    dfc['pv'] = dfc['pv'].fillna(0)  # nan을 0으로 채움
    dfc.fillna(0, inplace=True) # 남아있는 Nan을 0으로 입력
    return dfc

In [ ]:
# 데이터 읽기 & 데이터 보간
train = data_read('train-KEPCO') # 학습데이터 읽기
train

In [ ]:
train_pcon_max = train['pcon'].max() # 전력소비량의 최대값
train_pcon_min = train['pcon'].min() # 전력소비량의 최소값
train['pcon'] = (train['pcon'] - train_pcon_min)/(train_pcon_max-train_pcon_min) # 정규화

In [ ]:
train.head()

In [ ]:
plt.plot(train['pcon'])

In [ ]:
train.groupby(train['month']).mean()

In [ ]:
sub_train = train[ ['dayofweek', 'pcon']]
sub_train.groupby(sub_train['dayofweek']).sum()

In [ ]:
sub_train = train[ ['hour', 'pcon']]
sub_group = sub_train.groupby(sub_train['hour']).sum()

In [ ]:
plt.plot(sub_train['pcon'])

In [ ]:
sub_group

In [ ]:
plt.bar(sub_group.index, sub_group['pcon'])

In [ ]:
train_sample = train[ ['pcon', 'temp']]

In [ ]:
train_sample.corr()